## Bayesian Statistics, Markov-Chain Monte Carlo, and PyMC

#### Jack Bennetto
#### November 22, 2017

Last week we talked a bit about Bayesian statistics. To review:

Suppose we're considering some hypothesis $H$ and we've collected some data $\mathbf{X}$.
$$ P(H|\mathbf{X}) = \frac{P(\mathbf{X}|H) P(H)}{P(\mathbf{X})} $$

Each term has a name.

* $P(H)$ is the *prior probability*
* $P(\mathbf{X}|H)$ is the *likelihood*.
* $P(\mathbf{X})$ is the *normalizing constant*.
* $P(H|\mathbf{X})$ is the *posterior probability*.


If there are a bunch of hypotheses $H_1, H_2, ... H_n$, we could write this as

$$\begin{align}
P(H_i|\mathbf{X}) & = \frac{P(\mathbf{X}|H_i) P(H_i)}{P(\mathbf{X})}\\
         & = \frac{P(\mathbf{X}|H_i) P(H_i)}{\sum_{j=0}^{n} P(\mathbf{X}|H_j) P(H_j)}
\end{align}
$$

Here we see the normalizing constant is the likelihood times the prior summed over all possible hypothesis. In other words, it's the constant (independent of hypothesis) needed to be multiplied by all the numerators so that they all add up to one.



When you worked that out computationally you saw calculating the normalizing constant was difficult, particularly in cases with an infinite number of possible hypotheses. One approach we took was to used conjugate priors, but they aren't available in general. The other is to divide the hypothesis space into many slices, but that beocmes computationally impossible in high-dimensional space. In particular, that means giving the same attention to areas that are likely in the posterior as those that are almost impossible.

A third choice is random sampling. We call this a *monte-carlo* approach. In particular we sample many points (hypothese) from the posterior distribution, transitioning from each point to the next, in what is called Markov-Chain Monte Carlo (MCMC). There are a number of different approaches to this, but a common one, called the [Metropolis-Hastings](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm) algorithm, works like this.

First, we start at a given set of values. We then adjust them slightly (in such way that the probabilities are symmetric) and consider a new point. If it has a higher value for the pdf of the posterior, we accept it automatically. If it's lower, we accept it with a probability of the fraction of the ratio with the dpdf at the current point. Otherwise  we reject the new point. We repeat this many, mnay times.

Today most MCMC approaches use more sophisticated algorithms (pymc3 used NUTS (No-U-Turn Sampling)), but the idea is often similar.


First you need to install pymc3. Try using conda, like this:

`conda install -c conda-forge pymc3`

In [ ]:
import scipy.stats as scs
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Experimenting with pymc

Let's start with the coin-flip/CTR example we've seem a couple times before. In this we have data on successes and failures and we want to determine the value of some unknown click-through rate. First, let's make some data!

In [ ]:
prob_actual = 0.10
data_ctr = scs.bernoulli(prob_actual).rvs(100)
data_ctr

The pymc interface uses the `with` statement in python to specify a model with which we'll be operating. The first time we interact with in we'll need to call the constructor and assign it to a variable with `as`; after that we just refer to the variable. Any variables created are added to that model object automatically, and any sampling or optimization is done on that model.

In [ ]:
with pm.Model() as model_ctr:
    pass

In general, there are a couple different types of variables. First, we can create a variable that represents some prior distribution of a variable. Here we're just starting with a uniform distribution for a prior. Each object created in the model (usually) needs a name so we can identity it later. 

In [ ]:
with model_ctr:
    prob = pm.Uniform('prob', 0, 1)

The other type of variable in the model is a likelihood, specifying the observed data.

In [ ]:
with model_ctr:
    observed = pm.Bernoulli('observed', prob, observed=data_ctr)

Now we do some sampling! The result of sampling is often called a *trace*.

In [ ]:
with model_ctr:
    trace = pm.sample(10000)

The result of `sample` will contain values for all of the variables, distributed (if we have enough) in the same manner as the posterior distribution of those variables. Let's visualize them.

In [ ]:
fig, ax = plt.subplots()
ax.hist(trace['prob'], bins=50)
plt.show()

Questions:

Why doesn't it peak at the actual value for the probability (`prob_actual`)?

Why isn't this very smooth?

Is this a beta distribution?

What else can we do with this?

## A (slightly) harder example

Bernoulli distributions aren't all that exciting. Let's sample a few points from a normal distribution and  try to recover the parameters.

In [ ]:
mu_actual = 5
sigma_actual = 2
data = scs.norm(mu_actual, sigma_actual).rvs(10)

What do we have?

In [ ]:
print("mean = {:.3f} sd = {:.3f}".format(data.mean(), data.std()))

For now let's assume we know `sigma`, just to make it easier. We'll take a uniform prior for `mu` but assume it's between 0 and 10.

In [ ]:
with pm.Model() as model_normal:
    # prior
    mu = pm.Uniform("mu", 0, 10)
    # likelihood
    observed = pm.Normal("observed", mu, 2, observed=data)

We might want to start my calculating the maximum *a posteriori* (MAP) value for the `mu`.

Question: what is MAP?

In [ ]:
with model_normal:
    estimate = pm.find_MAP()
estimate

In [ ]:
with model_normal:
    trace = pm.sample(10000)

In [ ]:
fig, ax = plt.subplots()
ax.hist(trace['mu'], bins=50)
plt.show()

But really we want to estimate both the `mu` and `sigma`. We just need to put both in the model.

In [ ]:
with pm.Model() as model_normal2:
    # prior
    mu = pm.Uniform('mu', 0, 10)
    sigma = pm.Uniform('sigma', 0, 10)
    # likelihood
    observed = pm.Normal("observed", mu, sigma, observed=data)

In [ ]:
with model_normal2:
    estimate = pm.find_MAP()

Are these the right values?

In [ ]:
print("                   mu   sigma")
print("MAP estimate      {:5.3f} {:5.3f}".format(float(estimate['mu']), float(estimate['sigma'])))
print("sample statistics {:5.3f} {:5.3f}".format(data.mean(), data.std()))
print("actual            {:5.3f} {:5.3f}".format(mu_actual, sigma_actual))

In [ ]:
with model_normal2:
    trace = pm.sample(10000)

In [ ]:
fig, axes = plt.subplots(2)
axes[0].hist(trace['mu'], bins=50, normed=True)
axes[1].hist(trace['sigma'], bins=50, normed=True)
axes[0].set_title("mu")
axes[1].set_title("sigma")
plt.tight_layout()
plt.show()

There's a function to plot this automatically.

In [ ]:
pm.traceplot(trace)

A scatter plot will show us the how they are related.

In [ ]:
fig, ax = plt.subplots()
ax.plot(trace['mu'], trace['sigma'], '.', alpha=0.05)
ax.set_xlabel('mu')
ax.set_ylabel('sigma')
ax.plot(data.mean(), data.std(), 'rx', ms=20)
plt.show()

Note that these aren't independent. If sigma is small, then mu must be near the optimal value.

## Once more, together

So all that's great, but generally we're interested in doing more than taking the mean. Let's try doing a linear-regression problem together (with some fake data) before doing the assignment.